In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import io


file = os.path.join('..','data', 'pandas-post-dataset.csv')


dataset = pd.read_csv(file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")



FileNotFoundError: File b'../data/pandas-post-dataset.csv' does not exist

In [2]:



# For every question
#index, row in df.iterrows()
# for idx, q in high_level_questions.iterrows():
#     print('Working on the question %s' % q.Id)
#     # Extract every answer
#     answers = all_posts[(all_posts.ParentId == q.Id) & (all_posts.Score > ANSWER_SCORE)]
    

#     # Filter the code in the answer by using the <code> tag
#     unformatted_codes = []
#     for idx, a in answers.iterrows():
#         soup = BeautifulSoup(a['Body'], "html.parser")
#         unformatted_codes.extend(soup.find_all('code'))
    
#     # Format from HTML to text
#     codes = [i.get_text() for i in unformatted_codes]
    
#     # Write it to a file
#     file = io.open(os.path.join('..', 'data', 'extracted-code',  str(q.Id)), 'a', encoding='ISO-8859-1')
#     for code in codes:
#         file.write("%s\n\n------------------------------------------------------------------\n\n" % code)
#     file.close()                  





In [7]:
text3 = """
    <p>I have a dataframe <code>df</code> in pandas that was built using <code>pandas.read_table</code> from a csv file. The dataframe has several columns and it is indexed by one of the columns (which is unique, in that each row has a unique value for that column used for indexing.) </p>

<p>How can I select rows of my dataframe based on a ""complex"" filter applied to multiple columns? I can easily select out the slice of the dataframe where column <code>colA</code> is greater than 10 for example:</p>

<pre><code>df_greater_than10 = df[df[""colA""] &gt; 10]
</code></pre>

<p>But what if I wanted a filter like: select the slice of <code>df</code> where <em>any</em> of the columns are greater than 10? </p>

<p>Or where the value for <code>colA</code> is greater than 10 but the value for <code>colB</code> is less than 5?</p>

<p>How are these implemented in pandas?
Thanks.</p>
"""

def extract_tagged_code(text, tag):
    
    soup = BeautifulSoup(text, "html.parser")
    
    # Get the tag
    tagged = soup.find_all(tag)
    
    # Remove None Objects
#     tagged = [x for x in tagged if x is not None]
    
    # Format from HTML to text
    return [i.get_text() for i in tagged]

extract_tagged_code(text3, 'pre')
extract_tagged_code(text3, 'code')


['df',
 'pandas.read_table',
 'colA',
 'df_greater_than10 = df[df[""colA""] > 10]\n',
 'df',
 'colA',
 'colB']

In [5]:
def extract_code(df):
    """
        Extract only code with <pre><code> combined tag
    """
    codelist = extract_tagged_code(df['Body'], 'pre')
    df['CodeList'] = codelist
    df['Code'] = '\n'.join(codelist)
    
    return df


def extract_all_code(df):
    """
        Extract code with <code> combined tag
    """
    df['AllCode'] = extract_tagged_code(df['Body'], 'code')
    return df

# def extract_all_code(df):
#     """
#         Extract every code with <code> tag
#     """
#     soup = BeautifulSoup(df['Body'], "html.parser")
    
#     # Get the <code> tag
#     unformatted_codes = soup.find_all('code')
    
#     # Format from HTML to text
#     df['Code'] = [i.get_text() for i in unformatted_codes]
#     return df
    

# Extract Code
dataset = dataset.apply(extract_code, axis=1)
dataset = dataset.apply(extract_all_code, axis=1)

# Save to CSV
dataset_csv = dataset[['AnswerCount',
                                                 'CommentCount',
                                                 'Id',
                                                 'ParentId','PostTypeId','Score','Tags','Title', 'Body',
                                                 'ViewCount','Code', 'AllCode']]

dataset_csv.to_csv(os.path.join('..', 'data', 'pandas-code-dataset.csv'), encoding='ISO-8859-1', sep=";", doublequote=True)
# #high_level_questions_csv